# 多元统计分析作业一（小组）

- 胡义磊 2020010801005
- 张逸斐 2020010801006
- 阮登科 2020010801008
- 蔡与望 2020010801024

导入包，预设参数。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.cluster.hierarchy as sch
import seaborn as sns
from pyecharts.charts import Map
import pyecharts.options as opts

plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False
plt.tight_layout()

## 地图可视化

读取地区板块数据集，统计各地区上市公司数量。

In [ ]:
regions = pd.read_csv(
    "data/地区板块.txt",
    sep="\t",
    names=["地区代码", "地区名称", "股票代码", "股票名称"],
    dtype=str,
)
regions["地区名称"] = regions["地区名称"].apply(
    lambda name: str(name).replace("板块", "").replace("内蒙", "内蒙古")
)
region_name_counts = regions["地区名称"].value_counts()

在中国地图上展示热力图。

In [ ]:
Map().add(
    "上市数目",
    list(zip(region_name_counts.index, [int(value) for value in region_name_counts.values])),
    is_map_symbol_show=False,
    is_roam=False,
).set_global_opts(
    title_opts=opts.TitleOpts("各地区上市数目"),
    visualmap_opts=opts.VisualMapOpts(
        min_=0,
        max_=650,
        range_text=["上市最多", "上市最少"],
        is_piecewise=True,
        pos_top="middle",
        pos_left="left",
        orient="vertical",
        split_number=13,
    ),
).render_notebook()

## 地区行业分布

读取行业板块数据集，根据股票代码合并地区与行业两个数据集。

In [ ]:
industries = pd.read_csv(
    "data/行业板块.txt",
    sep="\t",
    names=["行业代码", "行业名称", "股票代码", "股票名称"],
    dtype=str,
)
region_industries = pd.merge(regions, industries, on="股票代码", how="inner")[["地区名称", "行业名称"]]
region_names = region_industries["地区名称"].unique()
industry_names = region_industries["行业名称"].unique()

绘制饼图，展示各地区各行业上市公司数量的占比。

In [ ]:
for region_name in region_names:
    current_region_industries = region_industries[region_industries["地区名称"] == region_name]
    industry_name_counts = current_region_industries["行业名称"].value_counts()
    industry_name_counts.plot(
        kind="pie",
        autopct=lambda pct: round(current_region_industries.shape[0] * pct / 100),
        title=f"{region_name}各行业上市数目",
        figsize=(16, 12),
    )
    plt.show()

## 聚类分析地区相似性

构造一个多维空间，其维数等于行业总数。每个点代表一个地区，其坐标代表该地区在各行业的上市公司数量。

In [ ]:
points = pd.DataFrame(0, index=region_names, columns=industry_names)
for region_name in region_names:
    current_region_industries = region_industries[region_industries["地区名称"] == region_name]
    industry_name_counts = current_region_industries["行业名称"].value_counts()
    points.loc[region_name, industry_name_counts.index] = industry_name_counts.values

根据地区间的欧氏距离，使用 Ward 法聚类。

In [ ]:
Z = sch.linkage(points, method="ward", metric="euclidean")
sch.dendrogram(Z, labels=points.index, orientation="left")
plt.show()

以 100 为距离阈值，所有地区被分为 4 类：

1. 深圳、广东、上海、山东
2. 北京
3. 浙江、江苏
4. 其余所有地区